In [39]:
import numpy as np
import pandas as pd
import xlrd
from tqdm import tqdm
from sklearn.linear_model import LinearRegression
from sklearn import preprocessing
from sklearn.preprocessing import Imputer

In [40]:
train_df=pd.read_excel("train.xlsx")

In [41]:
#### calculate miss values
def drop_missing_columns(train_df,num=50):
    col_missing_df = train_df.isnull().sum(axis=0).reset_index()
    col_missing_df.columns = ['col','missing_count']
    col_missing_df = col_missing_df.sort_values(by='missing_count')
    all_nan_columns = col_missing_df[col_missing_df.missing_count>num].col.values.tolist()
    train_df.drop(all_nan_columns,axis=1,inplace=True)
    return train_df
def split_columns(train_df):
    textcolumns=[]
    columns=train_df.columns.tolist()
    for item in columns:
        if item[0:1] not in ['0','1','2','3','4','5','6','7','8','9']:
            textcolumns.append(item)
    textcolumns.remove("Y")
    numcolumns=[col for col in columns if col not in textcolumns]
    return numcolumns,textcolumns
def Remove_duplicate_columns(train_df):
    return train_df.T.drop_duplicates().T
def drop_columns_all_equal(df):
    float_uniq_col = []
    for col in df.columns.values:
        uniq = df[col].unique()
        if len(uniq) == 1:
            float_uniq_col.append(col)
    df.drop(float_uniq_col,axis=0,inplace=True)
    return df
def date_cols(train_df):
    float_date_col = []
    for col in train_df.columns.tolist():
        if train_df[col].min() > 1e13:
            float_date_col.append(col)
    return float_date_col   
def build_model(x_train,y_train):
    reg_model = LinearRegression()
    reg_model.fit(x_train,y_train)
    return reg_model
def col_miss(train_df):
    col_missing_df = train_df.isnull().sum(axis=0).reset_index()
    col_missing_df.columns = ['col','missing_count']
    col_missing_df = col_missing_df.sort_values(by='missing_count')
    return col_missing_df

#### obtain cols of XX type
def obtain_x(train_df,xtype):
    dtype_df = train_df.dtypes.reset_index()
    dtype_df.columns = ['col','type']
    return dtype_df[dtype_df.type==xtype].col.values

def date_cols(train_df,float_col):
    float_date_col = []
    for col in float_col:
        if train_df[col].min() > 1e13:
            float_date_col.append(col)
    return float_date_col

def float_uniq(float_df,float_col):
    float_uniq_col = []
    for col in float_col:
        uniq = float_df[col].unique()
        if len(uniq) == 1:
            float_uniq_col.append(col)
    return float_uniq_col

def cal_corrcoef(float_df,y_train,float_col):
    corr_values = []
    for col in float_col:
        corr_values.append(abs(np.corrcoef(float_df[col].values,y_train)[0,1]))
    corr_df = pd.DataFrame({'col':float_col,'corr_value':corr_values})
    corr_df = corr_df.sort_values(by='corr_value',ascending=False)
    return corr_df


In [66]:
int64_col=obtain_x(train_df,'int64')

In [67]:
int64_df=Remove_duplicate_columns(train_df[int64_col])

In [68]:
int64_df

,210X17,210X19,210X21,210X24,210X31,210X36,210X37,210X38,210X77,210X80,...,520X152,520X173,520X246,520X250,520X346,520X348,520X351,520X353,520X375,520X377
0,750,2,14,20170725030628,1,450,0,0,96,0,...,20170724224422,20170724224112,69541,20170724224338,20170724224251,20170724224239,69525,69543,1850,1500
1,750,2,14,20170725030849,1,430,0,0,96,0,...,20170724224550,20170724224158,69542,20170724224456,20170724224408,20170724224356,69526,69544,1850,1500
2,750,2,14,20170725031113,1,425,0,0,96,0,...,20170724224717,20170724224244,69543,20170724224623,20170724224535,20170724224513,69527,69545,1850,1500
3,750,2,14,20170723105612,1,430,0,0,96,0,...,20170723090525,20170723084627,68971,20170723090341,20170723090253,20170723085325,68955,68973,1850,1500
4,750,2,14,20170723111008,1,430,0,0,96,0,...,20170723091457,20170723085422,68975,20170723091413,20170723091326,20170723091314,68959,68977,1850,1500
5,750,2,14,20170723115220,1,430,0,0,96,0,...,20170723095101,20170723093022,68987,20170723095017,20170723094929,20170723094917,68971,68989,1850,1500
6,750,2,14,20170723121001,1,450,0,0,96,0,...,20170723100144,20170723095143,68992,20170723100000,20170723095912,20170723095850,68976,68994,1850,1500
7,750,2,14,20170723093642,1,450,0,0,96,0,...,20170723054910,20170723053930,68904,20170723054719,20170723054631,20170723054619,68888,68906,1850,1500
8,750,2,14,20170723094350,1,450,0,0,96,0,...,20170723055626,20170723054555,68907,20170723055435,20170723055347,20170723055335,68891,68909,1850,1500
9,750,2,14,20170723095328,1,430,0,0,96,0,...,20170723061340,20170723055445,68911,20170723061157,20170723061110,20170723060142,68895,68913,1850,1500


In [69]:
uniq_col=float_uniq(int64_df,int64_df.columns.values)

In [71]:
good_columns=[item for item in int64_df.columns.tolist() if item not in uniq_col]

In [73]:
int64_df[good_columns].to_csv('int64.csv')

In [33]:
train_df = drop_missing_columns(train_df)
numcolumns,textcolumns=split_columns(train_df)
num_df=Remove_duplicate_columns(train_df[numcolumns])
num_df=drop_columns_all_equal(num_df)

In [34]:
datecol=date_cols(num_df)
for item in datecol:
    num_df.drop(item,axis=1,inplace=True)

In [36]:
train_df=train_df[num_df.columns.tolist()]
imputer=Imputer(strategy="median")
imputer.fit(train_df)
X=imputer.transform(train_df)
train_df= pd.DataFrame(X, columns=train_df.columns)
corr_matrix=train_df.corr()
df_corr=pd.DataFrame({'col':corr_matrix["Y"].index,'corr_value':corr_matrix["Y"].values})
corr02 = df_corr[df_corr.corr_value>=0.2]
corr02_col = corr02['col'].values.tolist()
train_df=train_df[corr02_col]

In [37]:
X=train_df.values

In [32]:
X = preprocessing.scale(X)

In [39]:
x_train = X[0:df1.shape[0]]
x_test = X[df1.shape[0]:]
y_train = df1.Y.values

In [42]:
model=build_model(x_train,y_train)

In [43]:
subA = model.predict(x_test)
    # read submit data
sub_df = pd.read_csv('subA.csv',header=None)
sub_df['Y'] = subA
sub_df.to_csv('github.csv',header=None,index=False)

In [44]:
from sklearn.model_selection import cross_val_score
lin_scores = cross_val_score(model, x_train, y_train,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)